# Lesson 2: Cleaning and Merging

[Previous Lesson](01-Requirements-Data-Identification.ipynb) [Next Lesson](02-Clean-Merge.ipynb)

## Contents


## Getting Started


In [66]:
import linecache
import random

def get_line_count(input_file):
    """Count number of lines in a file"""
    count = 0
    with open(input_file) as infile:
        for line in infile:
            count += 1
    return count

def sample_file(input_file, output_file, fraction=0.1):
    """Exctract a subset of lines from a file"""
    total_line_count = get_line_count(input_file)
    sample_line_count = int(fraction * total_line_count)
    sample_line_numbers = random.sample(range(1, total_line_count), sample_line_count)
    sample_line_numbers.sort()
    sample_line_numbers.insert(0, 0)
    with open(output_file, 'w') as outfile:
        for line_number in sample_line_numbers:
            line = linecache.getline(input_file, line_number + 1)
            outfile.write(line)
                    
        

In [49]:
from itertools import islice

with open("./data/02-franklin.csv") as infile:
    head = list(islice(infile, 5))

for line in head:
    print(line)

"PID","AEXMLND","AEXMBLD","AEXMTOT","APPRLND","APPRBLD","APPRTOT","AUDMAP","AUDRTG","LANDUSE","CAUV","SCHOOL","HOMSTD","MAILAD1","MAILAD2","MAILAD3","MAILAD4","TRANDT","TRANYR","NAME1","NAME2","NAME3","OWNER_ADD1","OWNER_ADD2","NBRHD","FLOOD","PCLASS","NOCARDS","ACREA","PRICE","ANN_TAX","STHNUM","STCONT","STHSFX","STDIRE","STNAME","STSFX","STADDR","USPS_CITY","STATE","ZIPCODE","DESCR1","DESCR2","DESCR3","TAXDESI","VALID","AREA_A","DWELTYP","ROOMS","BATHS","HBATHS","BEDRMS","AIRCOND","CINBRHD","COND","FIREPLC","GRADE","HEIGHT","NOSTORY","YEARBLT","PROPTYP","WALL","TIFMLND","TIFMBLD","POINT_X","POINT_Y"

"010-020705",0.00,0.00,0.00,8100.00,59600.00,67700.00,"D013",10.00,520,".00",2503,,"MIDWEST LOAN SERVICES",,"616 SHELDEN AVE STE 300","HOUGHTON MI 49931-1841",3/22/2016 0:00:00,2016,"NORRIS ROBIN R","HUGO ROBIN",,"246 ELDON AVE","COLUMBUS OH         43204",94.00,,"R","1",0.00,0.00,1455.02,"244","-","246","N","ELDON","AV","244 - 246 N ELDON AV","COLUMBUS","OH",43204,"244-6 ELDON AVE","HIL

In [50]:
import pandas as pd
franklin = pd.read_csv('./data/02-franklin.csv')

In [52]:
franklin.head()

,PID,AEXMLND,AEXMBLD,AEXMTOT,APPRLND,APPRBLD,APPRTOT,AUDMAP,AUDRTG,LANDUSE,...,GRADE,HEIGHT,NOSTORY,YEARBLT,PROPTYP,WALL,TIFMLND,TIFMBLD,POINT_X,POINT_Y
0,010-020705,0.0,0.0,0.0,8100.0,59600.0,67700.0,D013,10.0,520,...,C,DUPLEX,20.0,1925.0,NaN,WD/ALUM/VINYL,0.0,0.0,1809670.02,714541.95
1,010-020716,0.0,0.0,0.0,4600.0,69800.0,74400.0,D021,11.0,510,...,C,OLD STYLE,20.0,1919.0,NaN,WD/ALUM/VINYL,0.0,0.0,1809445.97,714129.83
2,010-020713,0.0,0.0,0.0,4900.0,60600.0,65500.0,D013,18.0,510,...,C,OLD STYLE,20.0,1925.0,NaN,WD/ALUM/VINYL,0.0,0.0,1809641.41,714204.64
3,010-020784,0.0,0.0,0.0,5800.0,0.0,5800.0,D004,12.0,510,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1809302.85,715262.74
4,010-020773,0.0,0.0,0.0,5000.0,31200.0,36200.0,D004,8.0,510,...,C-1,RANCH,10.0,1922.0,NaN,WD/ALUM/VINYL,0.0,0.0,1808870.83,715286.02


In [53]:
franklin.columns.tolist()

['PID',
 'AEXMLND',
 'AEXMBLD',
 'AEXMTOT',
 'APPRLND',
 'APPRBLD',
 'APPRTOT',
 'AUDMAP',
 'AUDRTG',
 'LANDUSE',
 'CAUV',
 'SCHOOL',
 'HOMSTD',
 'MAILAD1',
 'MAILAD2',
 'MAILAD3',
 'MAILAD4',
 'TRANDT',
 'TRANYR',
 'NAME1',
 'NAME2',
 'NAME3',
 'OWNER_ADD1',
 'OWNER_ADD2',
 'NBRHD',
 'FLOOD',
 'PCLASS',
 'NOCARDS',
 'ACREA',
 'PRICE',
 'ANN_TAX',
 'STHNUM',
 'STCONT',
 'STHSFX',
 'STDIRE',
 'STNAME',
 'STSFX',
 'STADDR',
 'USPS_CITY',
 'STATE',
 'ZIPCODE',
 'DESCR1',
 'DESCR2',
 'DESCR3',
 'TAXDESI',
 'VALID',
 'AREA_A',
 'DWELTYP',
 'ROOMS',
 'BATHS',
 'HBATHS',
 'BEDRMS',
 'AIRCOND',
 'CINBRHD',
 'COND',
 'FIREPLC',
 'GRADE',
 'HEIGHT',
 'NOSTORY',
 'YEARBLT',
 'PROPTYP',
 'WALL',
 'TIFMLND',
 'TIFMBLD',
 'POINT_X',
 'POINT_Y']

In [65]:
from IPython.display import IFrame
IFrame("./data/02-franklin-description.pdf", 800, 600)


In [69]:
from itertools import islice

with open("./data/02-licking.txt") as infile:
    head = list(islice(infile, 5))

for line in head:
    print(line)

fldParcelID;fldParcelNo;fldRoutingNo;fldOwner;fldCardNo;fldMaxCards;fldRecheck;fldSchoolDistrict;fldTaxDistrict;fldLegalDesc;fldLocationAddress;fldNeighborhood;fldPropertyType;fldLUC;fldTopo;fldAccess;fldUtilities;fldNotes;fldNotes2;fldMarketLand;fldMarketImprov;fldMarketTotal;fldCAUVLand;fldCAUVImprov;fldCAUVTotal;fldSketchPath;fldStyle;fldStories;fldExterior;fldHeating;fldCooling;fldBasement;fldFullBaths;fldHalfBaths;fldOtherBaths;fldAttic;fldAtticHeating;fldRooms;fldBedrooms;fldFamilyRooms;fldDiningRooms;fldLivingRooms;fldFinishedLivingArea;fldFinishedBasementArea;fldYearBuilt;fldEffYearBuilt;fldYearRemodeled;fldCondition;fldGrade;fldBasementGarage;fldFireplaceOpenings;fldFireplaceStacks;fldFirstFloorArea;fldFirstFloorCost;fldUpperFloorArea;fldUpperFloorCost;fldAtticArea;fldAtticCost;fldHalfFloorArea;fldHalfFloorCost;fldFinishedBasementCost;fldUnfinishedLivingArea;fldUnfinishedLivingCost;fldCrawlArea;fldCrawlCost;fldBasementArea;fldBasementCost;fldBasementGarageArea;fldBasementGarag

In [70]:
licking = pd.read_csv("./data/02-licking.txt")

ParserError: Error tokenizing data. C error: Expected 33 fields in line 3, saw 34


In [71]:
licking = pd.read_csv("./data/02-licking.txt", delimiter=";")

ParserError: Error tokenizing data. C error: Expected 181 fields in line 1608, saw 182


In [81]:
typical_line = linecache.getline("./data/02-licking.txt", 2)
error_line = linecache.getline("./data/02-licking.txt", 1608)

In [82]:
print(typical_line)
print(error_line)

35;001-000066-00.009;001-006.00-120.000;HARDY RONALD J & LINDA B;1;1;No;NORTHRIDGE LSD;001 - BENNINGTON T-NRDGE LSD;LOT 10 PT   3.68 AC;8713 BENNER RD;06700 Bennington-T;Dwelling;511 Single family unplatted 0-09.9;;;2578 Electric Gas Well Septic;;;50300;138100;188400;0;0;0;C:\tempPRCsketch.bmp;Single Family;1;Frame;Central Warm Air;Central;Full Basement;2.0;;;None;;6.0;3.0;;0.0;;1,790;0;2002;2002;;Average;90;;;;1,790;115,400;0;0;0;0;0;0;0;0;0;0;0;1,790;30,600;0;0;;0;;0;;5,400;;4,500;S(0), O(0);0;1;1;;15,400;;0;171,300;90;-17,130;100;0;;;166,200;-33,240;No;20;0;0.00000;;133,000;5,100;138,100;Well/Septic;12,000;2016;2015;2014;41,300;41,300;41,300;141,500;141,500;141,500;182,800;182,800;182,800;06/07/2001;11/01/2000;07/06/1999;;1;4;4;;FD - FIDUCIARY;EX - EXEMPT CONVEYANCE;EX - EXEMPT CONVEYANCE;;32000.00;0.00;0.00;;01721     ;99999     ;99999     ;;Y;N;N;;Y;Y;Y;;BEVIER MARLENE M TRUSTEE;SHARROCK LORI S TRUSTEE;Refer to deed;;;50,300;5,100;15,400;2017;146,000;No;No;No;No;No;0;100;138,100;0

In [83]:
print(typical_line.count(";"))
print(error_line.count(";"))

180
181


In [86]:
from itertools import zip_longest

header_line = linecache.getline("./data/02-licking.txt", 1)

header_entries = header_line.split(";")
typical_entries = typical_line.split(";")
error_entries = error_line.split(";")

for entry in zip_longest(header_entries, typical_entries, error_entries):
    print(entry)

('fldParcelID', '35', '14934')
('fldParcelNo', '001-000066-00.009', '020-051654-00.000')
('fldRoutingNo', '001-006.00-120.000', '020-116.18-017.000')
('fldOwner', 'HARDY RONALD J & LINDA B', 'FULLER JEAN A TRUSTEE')
('fldCardNo', '1', '1')
('fldMaxCards', '1', '1')
('fldRecheck', 'No', 'No')
('fldSchoolDistrict', 'NORTHRIDGE LSD', 'GRANVILLE EVSD')
('fldTaxDistrict', '001 - BENNINGTON T-NRDGE LSD', '020 - GRANVILLE T - GRANVILLE V&EVSD')
('fldLegalDesc', 'LOT 10 PT   3.68 AC', 'LOT 116')
('fldLocationAddress', '8713 BENNER RD', '128 E BROADWAY')
('fldNeighborhood', '06700 Bennington-T', '05500 Granville-Vil-West Half')
('fldPropertyType', 'Dwelling', 'Building')
('fldLUC', '511 Single family unplatted 0-09.9', '430 Resturant')
('fldTopo', '', ' cafteria and/or bar')
('fldAccess', '', '')
('fldUtilities', '2578 Electric Gas Well Septic', '')
('fldNotes', '', '6 All')
('fldNotes2', '', '')
('fldMarketLand', '50300', '')
('fldMarketImprov', '138100', '165100')
('fldMarketTotal', '188400',

In [122]:
licking = pd.read_csv("./data/02-licking.txt", delimiter="(?<!Resturant);", engine="python")

In [123]:
licking.shape

(8363, 181)

In [134]:
# problem is "fldLUC: 430 Resturant; cafteria and/or bar"
# could use error_bad_lines with read_csv
# could use regex separator

# difference in dtypes due to 0 as float vs int and NaN
import io

class StringIOWithReplace(io.FileIO):
    def __init__(self, path, old_value, new_value):
        super(StringIOWithReplace, self).__init__(path)
        self._old_value = old_value.encode()
        self._new_value = new_value.encode()
    
    def read(self, *args, **kwargs):
        return super(StringIOWithReplace, self).read(*args, **kwargs).replace(self._old_value, self._new_value)
        

            
licking_file = StringIOWithReplace("./data/02-licking.txt", "Resturant; cafteria", "Resturant cafteria")
licking2 = pd.read_csv(licking_file, delimiter=";")
    



/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (56,58,61,89,132,133,134,159,163,164,165) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [126]:
licking.columns[56], licking2.columns[56]

('fldAtticArea', 'fldAtticArea')

In [127]:
licking.fldAtticArea

0         0
1       616
2         0
3         0
4         0
5         0
6       NaN
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18      NaN
19      NaN
20      NaN
21        0
22        0
23      104
24        0
25        0
26        0
27        0
28      NaN
29        0
       ... 
8333    NaN
8334    NaN
8335    NaN
8336      0
8337      0
8338      0
8339      0
8340    NaN
8341    NaN
8342    NaN
8343      0
8344    NaN
8345    NaN
8346      0
8347      0
8348      0
8349      0
8350      0
8351      0
8352    NaN
8353      0
8354      0
8355      0
8356      0
8357      0
8358      0
8359    NaN
8360    NaN
8361    NaN
8362    NaN
Name: fldAtticArea, Length: 8363, dtype: object

In [128]:
licking2.fldAtticArea

0         0
1       616
2         0
3         0
4         0
5         0
6       NaN
7         0
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18      NaN
19      NaN
20      NaN
21        0
22        0
23      104
24        0
25        0
26        0
27        0
28      NaN
29        0
       ... 
8333    NaN
8334    NaN
8335    NaN
8336      0
8337      0
8338      0
8339      0
8340    NaN
8341    NaN
8342    NaN
8343      0
8344    NaN
8345    NaN
8346      0
8347      0
8348      0
8349      0
8350      0
8351      0
8352    NaN
8353      0
8354      0
8355      0
8356      0
8357      0
8358      0
8359    NaN
8360    NaN
8361    NaN
8362    NaN
Name: fldAtticArea, Length: 8363, dtype: object

In [133]:
licking.fldAtticArea.iloc[8358],licking2.fldAtticArea.iloc[8358]

('0', 0.0)